# Thresholding Updraft Helicity

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import xoak
import geopandas as gpd
import regionmask

import warnings
warnings.filterwarnings('ignore')

In [2]:
#find directories with the WRF-BCC data
ref_dirts = glob.glob('/home/scratch/WRF_BCC/reflectivity/REFD/mid_century_8p5_FIXED/*')
ref_dirts.sort()
ref_dirts = ref_dirts[:]

uh_dirts = glob.glob('/home/scratch/WRF_BCC/severe_weather/UP_HELI_MAX/mid_century_8p5_FIXED/*')
uh_dirts.sort()

### Mask US Data

In [3]:
#load the WRF-BCC geog and a random refc dummy file
geog = xr.open_dataset("/home/scratch/WRF_BCC/geography/geo_em.d01.nc")
ds = xr.open_mfdataset('/home/scratch/WRF_BCC/reflectivity/REFD/historical/1990-1991/*.nc')

#merge the files and create needed infomation
ds = xr.merge([ds, geog.squeeze()])
ds = ds.rename({"CLONG": 'lon', 'CLAT': 'lat'})
ds = ds.assign_coords({'x': ds.west_east, 'y': ds.south_north})
ds = ds.assign_coords({'lon': ds.lon, 'lat': ds.lat})

#set the lat-lon as the index
ds.xoak.set_index(['lat', 'lon'], 'sklearn_geo_balltree')

In [4]:
#load an USA shapefile
usa = gpd.read_file("/home/jcorner1/Unidata/shapefiles/smoothing_econus.shp")

#mask the data out
state_mask = regionmask.mask_geopandas(usa, ds.lon, ds.lat)
ma = state_mask.values
ma[~np.isnan(ma)] = 1

### Threshold Data

In [ ]:
#iterate through each year (directory)
for dirt_number in range(len(ref_dirts)):
       for month in [10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    
        #open all the data within the directory
        print(f'current year: {int(ref_dirts[dirt_number][-4:])-1}, {month}')
        ref_ds = xr.open_mfdataset(glob.glob(f'{ref_dirts[dirt_number]}/*{ref_dirts[dirt_number][-4:]}_*-{str(month).zfill(2)}-*.nc'))
        uh_ds = xr.open_mfdataset(glob.glob(f'{uh_dirts[dirt_number]}/*-{str(month).zfill(2)}-*.nc'))

        #grab all values
        refc_val = ref_ds.REFD.values
        uh_val = uh_ds.UP_HELI_MAX.values

        #subset times
        times = ref_ds.Time.values

        #threshold upward vertical velocities, reflectivity, and updraft helicity values. 
        thr_uh = uh_ds.where(uh_val >= 35.0, 0)
        thr_uh = thr_uh.where(thr_uh.W_UP_MAX.values <= 35.0, 1)

        #Give all areas with a UH of 18+ and reflectivity of 50+ a value of 1. 
        thr_val = thr_uh.W_UP_MAX.values * ma

        #find locations where the value is 1.
        locations = np.where(thr_val >= 1)
        print(f'done thresholding! {len(locations[0])} potential storms')

        #close files
        thr_uh.close()
        uh_ds.close()

        #create pandas dataframe
        df = pd.DataFrame(columns=['x', 'y', 'Time', 'UH'])

        xs = []
        ys = []
        timess=[]
        refcs=[]
        uhs=[]

        #iterate through all potential center points. 
        for point in range(len(locations[0])):

            #save important attribute values for center points
            time = locations[0][point]
            y = locations[1][point] 
            x = locations[2][point]

            #append values to a list
            str_time = np.datetime_as_string(times[time])
            xs.append(x)
            ys.append(y)
            timess.append(str_time)
            refcs.append(refc_val[time,y,x])
            uhs.append(uh_val[time,y,x])

            #df = df.append({'x': x, 'y':y, 'Time':str_time, 'DBZ': refc_val[time,y,x]}, ignore_index=True)

        #add list to pandas dataframe
        df['x'] = xs
        df['y'] = ys
        df['Time'] = timess
        df['DBZ'] = refcs
        df['UH'] = uhs

        #Save the dataframe as the csv.
        df.to_csv(f'/home/scratch/jcorner1/syn_sev/dataframes/MC8p5_UH_{str_time[:4]}_{month}_threshold_dataframe.csv')



current year: 2040, 10
done thresholding! 6067 potential storms
current year: 2040, 11
done thresholding! 470 potential storms
current year: 2040, 12
done thresholding! 10811 potential storms
current year: 2040, 1
done thresholding! 340 potential storms
current year: 2040, 2
done thresholding! 3365 potential storms
current year: 2040, 3
done thresholding! 29476 potential storms
current year: 2040, 4
done thresholding! 10761 potential storms
current year: 2040, 5
done thresholding! 66956 potential storms
current year: 2040, 6
done thresholding! 60979 potential storms
current year: 2040, 7
done thresholding! 58502 potential storms
current year: 2040, 8
done thresholding! 28582 potential storms
current year: 2040, 9
done thresholding! 11940 potential storms
current year: 2041, 10
done thresholding! 10454 potential storms
current year: 2041, 11
done thresholding! 624 potential storms
current year: 2041, 12
done thresholding! 96 potential storms
current year: 2041, 1
done thresholding! 1616